<a href="https://colab.research.google.com/github/MaliheDahmardeh/World-Development-Indicators/blob/main/WDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The "World Development Indicators" from the World Bank contain over a thousand annual indicators of economic development from hundreds of countries around the world that is from 1960 to 2016. This dataset contains over 1,500 indicators related to various social and economic issues across countries and regions of the world. Some of the indicators related to social policies and issues include:

- Poverty headcount ratio
- Access to improved water sources
- Access to improved sanitation facilities
- Infant mortality rate
- Maternal mortality rate
- Primary completion rate
- Secondary completion rate
- Tertiary completion rate
- Unemployment rate
- Income share held by lowest 20%

There are also indicators related to crime rates and technology issues, such as:

- Homicide rate
- Intentional homicides per 100,000 people
- Mobile cellular subscriptions per 100 people
- Internet users per 100 people

Using this dataset, one could conduct various correlation analyses between population rates and these social and economic indicators.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from plotly.offline import download_plotlyjs,init_notebook_mode, plot
init_notebook_mode(connected=True)
# cf.go_offline()
import warnings
warnings.filterwarnings('ignore')

#Import DataSet

In [45]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir "/content/drive/My Drive/data"

import os
os.listdir('/content/drive/MyDrive/data')

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/data’: File exists


['Country.csv', 'Indicators.csv']

In [46]:
import pandas as pd

path = '/content/drive/MyDrive/data/Indicators.csv'
df = pd.read_csv(path)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656458 entries, 0 to 5656457
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CountryName    object 
 1   CountryCode    object 
 2   IndicatorName  object 
 3   IndicatorCode  object 
 4   Year           int64  
 5   Value          float64
dtypes: float64(1), int64(1), object(4)
memory usage: 258.9+ MB


In [48]:
df.isnull().sum()

CountryName      0
CountryCode      0
IndicatorName    0
IndicatorCode    0
Year             0
Value            0
dtype: int64

In [49]:
df.shape

(5656458, 6)

In [51]:
# How many unique country codes and indicator names are there ? 
countries = df['CountryCode'].unique().tolist()
indicators = df['IndicatorName'].unique().tolist()
print('How many unique country codes and indicator names are there?')
print('Contries size: %d' % len(countries))
print('Indicators size: %d' % len(indicators))

How many unique country codes and indicator names are there?
Contries size: 247
Indicators size: 1344
